#### SQLite : Tabellen löschen (DROP TABLE IF EXISTS ...)

In [2]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn


def drop_table(conn, sql):
    try:
        c = conn.cursor()
        c.execute(sql)
    except Error as e:
        print(e)


def main():
    database = 'O:/Z_Download/test_SQLite.db'
    # database = r'O:\01_Daten\DatenLocal\Datenbanken\SQLite\treibstoffpreise.db'
    sql_drop_tankstellen_table = """
    DROP TABLE IF EXISTS tankstellen;
    """
    sql_drop_treibstoffe_table = """
    DROP TABLE IF EXISTS treibstoffe;
    """
    sql_drop_preise_table = """
    DROP TABLE IF EXISTS preise;
    """
    conn = create_connection(database)
    if conn is not None:
        with conn:
            drop_table(conn, sql_drop_tankstellen_table)
            drop_table(conn, sql_drop_treibstoffe_table)
            drop_table(conn, sql_drop_preise_table)
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()


#### SQLite : Tabellen anlegen (CREATE TABLE IF NOT EXISTS ...)

In [ ]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def main():
    database = 'O:/Z_Download/test_SQLite.db'
    sql_create_tankstellen_table = """
    CREATE TABLE IF NOT EXISTS tankstellen (
        id      INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        Url     TEXT    NOT NULL
                        UNIQUE ON CONFLICT ROLLBACK,
        Name    TEXT    NOT NULL,
        Plz     TEXT    NOT NULL,
        Stadt   TEXT    NOT NULL,
        Strasse TEXT    NOT NULL
    );
    """
    sql_create_treibstoffe_table = """
    CREATE TABLE IF NOT EXISTS treibstoffe (
        id          INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        Bezeichnung TEXT    UNIQUE ON CONFLICT ROLLBACK
                            NOT NULL
    );
    """
    sql_create_preise_table = """
    CREATE TABLE IF NOT EXISTS preise (
        id            INTEGER PRIMARY KEY ASC AUTOINCREMENT,
        tankstelle_id INTEGER NOT NULL,
        treibstoff_id INTEGER NOT NULL,
        timestamp     DECIMAL,
        preis         DECIMAL,
        datum_zeit    TEXT,
        FOREIGN KEY (
            tankstelle_id
        )
        REFERENCES tankstellen (id),
        FOREIGN KEY (
            treibstoff_id
        )
        REFERENCES treibstoffe (id) 
    );
    """
    conn = create_connection(database)
    if conn is not None:
        with conn:
            create_table(conn, sql_create_tankstellen_table)
            create_table(conn, sql_create_treibstoffe_table)
            create_table(conn, sql_create_preise_table)
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()


#### SQLite : Datensätze eintragen (INSERT ...)

In [1]:
import sqlite3
from sqlite3 import Error
from rwm_mod01 import datum_zu_ts


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn


def create_tankstelle(conn, tankstelle):
    sql = ''' INSERT INTO tankstellen(Url,Name,Plz,Stadt,Strasse)
              VALUES(?,?,?,?,?) '''
    print(f'Tankstelle : {tankstelle}')
    cur = conn.cursor()
    try:
        cur.execute(sql, tankstelle)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(f'Tankstelle existiert schon : {e}')
        return None


def create_treibstoff(conn, treibstoff):
    sql = ''' INSERT INTO treibstoffe(Bezeichnung)
              VALUES(?) '''
    print(f'Treibstoff : {treibstoff}')
    cur = conn.cursor()
    try:
        cur.execute(sql, treibstoff)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(f'Treibstoff existiert schon : {e}')
        return None


def create_preis(conn, preis):
    sql = ''' INSERT INTO preise(tankstelle_id,treibstoff_id,timestamp,preis,datum_zeit)
              VALUES(?,?,?,?,?) '''
    print(f'Preis : {preis}')
    cur = conn.cursor()
    try:
        cur.execute(sql, preis)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(f'Preiseintrag existiert schon : {e}')
        return None


def main():
    database = 'O:/Z_Download/test_SQLite.db'
    # conn = create_connection(database)
    with create_connection(database) as conn:
        tankstelle = ('12345#', 'AGIP', '73733',
                      'Esslingen', 'Lerchenbergstr. 94')
        tankstelle_id = create_tankstelle(conn, tankstelle)
        print(f'tankstelle_id = {tankstelle_id}')
        treibstoff = ('Spezial Sprit',)
        treibstoff_id = create_treibstoff(conn, treibstoff)
        print(f'treibstoff_id = {treibstoff_id}')
        preis = (1, 1, datum_zu_ts('16.03.2022 17:19:00'), 2.239, '16.03.2022 17:19:00')
        preis_id = create_preis(conn, preis)


if __name__ == '__main__':
    main()


Tankstelle : ('12345#', 'AGIP', '73733', 'Esslingen', 'Lerchenbergstr. 94')
Tankstelle existiert schon : UNIQUE constraint failed: tankstellen.Url
tankstelle_id = None
Treibstoff : ('Spezial Sprit',)
Treibstoff existiert schon : UNIQUE constraint failed: treibstoffe.Bezeichnung
treibstoff_id = None
Preis : (1, 1, 1647447540.0, 2.239)


#### SQLite : Datensätze auslesen (SELECT ...)

In [35]:
import sqlite3
from sqlite3 import Error
from rwm_mod01 import ts_zu_datum


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn


def main():
    database = 'O:/Z_Download/test_SQLite.db'
    conn = create_connection(database)
    with conn:
        # conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        spalte = "12345#"
        sql = f''' SELECT * FROM tankstellen WHERE Url = "{spalte}" '''
        print(sql)
        datensatz = cur.execute(sql).fetchone()
        print(type(datensatz))
        print(f'datensatz = {datensatz}')
        # tankstellen_id = datensatz['id']
        if datensatz != None:
            print(f'tankstellen_id = {datensatz[0]}')
        else:
            print(f'Datensatz nicht vorhanden')


if __name__ == '__main__':
    main()


 SELECT * FROM tankstellen WHERE Url = "12345#" 
<class 'tuple'>
datensatz = (1, '12345#', 'AGIP', '73733', 'Esslingen', 'Lerchenbergstr. 94')
tankstellen_id = 1


#### SQLite : Alle Datensätze auslesen (SELECT ...)

In [28]:
import sqlite3
from sqlite3 import Error
from rwm_mod01 import ts_zu_datum


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn


def main():
    database = 'O:/Z_Download/test_SQLite.db'
    conn = create_connection(database)
    with conn:
        cur = conn.cursor()
        sql_anweisung = """
        select
        tankstellen.Name, treibstoffe.Bezeichnung,
        preise.preis, preise.timestamp
        from
        tankstellen, treibstoffe, preise
        where
        (treibstoffe.id = preise.treibstoff_id) and
        (tankstellen.id = preise.tankstelle_id)
        ;"""
        cur.execute(sql_anweisung)
        rows = cur.fetchall()
    for row in rows:
        row = [n for n in row]
        row[3] = ts_zu_datum(row[3])
        print(row)


if __name__ == '__main__':
    main()


['AGIP', 'Spezial Sprit', 2.239, '16.03.2022 17:19:00']
['AGIP', 'Spezial Sprit', 2.239, '16.03.2022 17:19:00']
['AGIP', 'Spezial Sprit', 2.239, '16.03.2022 17:19:00']
['AGIP', 'Spezial Sprit', 2.239, '16.03.2022 17:19:00']
